# Semeval 2025 Task 10
### Subtask 2: Narrative Baseline Classification -- Multilingual

Given a news article and a [two-level taxonomy of narrative labels](https://propaganda.math.unipd.it/semeval2025task10/NARRATIVE-TAXONOMIES.pdf) (where each narrative is subdivided into subnarratives) from a particular domain, assign to the article all the appropriate subnarrative labels. This is a multi-label multi-class document classification task.

## 1. Setup

### 1.1 Getting and analyzing data

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

from matplotlib import pyplot as plt
import seaborn as sns
import os

In [2]:
data = []
ignore_folders = ['.DS_Store']

base_dir_documents = '../data/semeval_data/train/raw-documents'

for language_folder in os.listdir(base_dir_documents):

    if language_folder in ignore_folders:
        continue

    language_path = os.path.join(base_dir_documents, language_folder)
    if os.path.isdir(language_path):
        for root, _, files in os.walk(language_path):
            for file in files:
                if file.endswith('.txt'):
                    file_path = os.path.join(root, file)

                    article_id = file
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()

                    data.append({
                        'language': language_folder,
                        'article_id': article_id,
                        'content': content
                    })

documents_df = pd.DataFrame(data)

In [3]:
print(documents_df.shape)
documents_df.head()

(1709, 3)


,language,article_id,content
0,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...
1,RU,RU-URW-1175.txt,В ЕС испугались последствий популярности правы...
2,RU,RU-URW-1149.txt,Возможность признания Аллы Пугачевой иностранн...
3,RU,RU-URW-1015.txt,Азаров рассказал о смене риторики Киева по пер...
4,RU,RU-URW-1001.txt,В россиянах проснулась массовая любовь к путеш...


In [4]:
base_dir_labels = '../data/semeval_data/train/labels'

raw_annotation_data = []

for language_folder in os.listdir(base_dir_labels):

    if language_folder in ignore_folders:
        continue

    print('Now processing language', language_folder)

    language_path = os.path.join(base_dir_labels, language_folder)
    if os.path.isdir(language_path):
        for root, _, files in os.walk(language_path):
            label_file = 'subtask-2-annotations.txt'
            file_path = os.path.join(root, label_file)

            with open(file_path, 'r') as file:
                for line in file:
                    parts = line.strip().split('\t')
                    article_id = parts[0]
                    narrative_to_subnarratives = parts[2].split(';')
                    narratives = []
                    subnarratives = []

                    for nar_to_sub in narrative_to_subnarratives:
                      subnarrative_list = nar_to_sub.split(' ')
                      if subnarrative_list[0] == 'Other':
                        narratives.append('Other')
                        subnarratives.append('Other')
                        continue

                      nar_to_sub = ' '.join(subnarrative_list[1:])
                      nar, sub = nar_to_sub.split(':')
                      narratives.append(nar.strip())
                      subnarratives.append(sub.strip())

                    raw_annotation_data.append({
                        'article_id': article_id,
                        'narratives': narratives,
                        'subnarratives': subnarratives
                    })

annotations_df = pd.DataFrame(raw_annotation_data)

Now processing language RU
Now processing language PT
Now processing language BG
Now processing language HI
Now processing language EN


In [5]:
annotations_df.head()

,article_id,narratives,subnarratives
0,RU-URW-1080.txt,[Discrediting Ukraine],[Discrediting Ukrainian government and officia...
1,RU-URW-1013.txt,"[Discrediting the West, Diplomacy]","[The West does not care about Ukraine, only ab..."
2,RU-URW-1145.txt,[Praise of Russia],[Praise of Russian military might]
3,RU-URW-1048.txt,[Discrediting Ukraine],[Discrediting Ukrainian military]
4,RU-URW-1001.txt,[Praise of Russia],[Russia is a guarantor of peace and prosperity]


In [6]:
annotations_df.tail()

,article_id,narratives,subnarratives
1694,EN_CC_200022.txt,"[Criticism of institutions and authorities, Cr...","[Criticism of national governments, Other, Met..."
1695,EN_CC_100028.txt,[Other],[Other]
1696,EN_CC_300010.txt,[Amplifying Climate Fears],[Other]
1697,EN_UA_013257.txt,"[Russia is the Victim, Blaming the war on othe...",[Russia actions in Ukraine are only self-defen...
1698,EN_UA_000104.txt,[Other],[Other]


In [7]:
annotations_df.shape

(1699, 3)

In [8]:
dataset = pd.merge(documents_df, annotations_df, on='article_id')
dataset.head()

,language,article_id,content,narratives,subnarratives
0,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,[Blaming the war on others rather than the inv...,"[The West are the aggressors, Other, The West ..."
1,RU,RU-URW-1175.txt,В ЕС испугались последствий популярности правы...,"[Discrediting the West, Diplomacy, Discreditin...","[The West is weak, Other, The EU is divided]"
2,RU,RU-URW-1149.txt,Возможность признания Аллы Пугачевой иностранн...,[Distrust towards Media],[Western media is an instrument of propaganda]
3,RU,RU-URW-1015.txt,Азаров рассказал о смене риторики Киева по пер...,"[Discrediting Ukraine, Discrediting Ukraine]","[Ukraine is a puppet of the West, Discrediting..."
4,RU,RU-URW-1001.txt,В россиянах проснулась массовая любовь к путеш...,[Praise of Russia],[Russia is a guarantor of peace and prosperity]


In [9]:
dataset.shape

(1699, 5)

In [10]:
dataset.head()

,language,article_id,content,narratives,subnarratives
0,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,[Blaming the war on others rather than the inv...,"[The West are the aggressors, Other, The West ..."
1,RU,RU-URW-1175.txt,В ЕС испугались последствий популярности правы...,"[Discrediting the West, Diplomacy, Discreditin...","[The West is weak, Other, The EU is divided]"
2,RU,RU-URW-1149.txt,Возможность признания Аллы Пугачевой иностранн...,[Distrust towards Media],[Western media is an instrument of propaganda]
3,RU,RU-URW-1015.txt,Азаров рассказал о смене риторики Киева по пер...,"[Discrediting Ukraine, Discrediting Ukraine]","[Ukraine is a puppet of the West, Discrediting..."
4,RU,RU-URW-1001.txt,В россиянах проснулась массовая любовь к путеш...,[Praise of Russia],[Russia is a guarantor of peace and prosperity]


In [11]:
row = 5
english_article = dataset[dataset['language'] == 'EN'].iloc[row].content
english_article

'Trump Lawyer Demands Accountability From Intel Chiefs Who Backed Hunter Biden \n\n An attorney for former President Donald Trump wants the 51 former intelligence chiefs held responsible for backing Hunter Biden in the unfolding story of the laptop abandoned in a Delaware repair shop.\n\nLawyer Tim Parlatore\'s goal is to uncover alleged communications between the 51 former senior intel leaders and the Biden 2020 campaign.\n\nPolitico had reported that an Oct. 19, 2020, letter, signed by the former intelligence officials, outlined their assessment that a New York Post disclosure of emails allegedly belonging to Hunter Biden "has all the classic earmarks of a Russia information operation."\n\nThose signing the letter included former CIA Directors Leon Panetta, Mike Hayden and John Brennan, along with former Director of National Intelligence James Clapper.\n\nThe letter offered no evidence, but raised suspicions by the former intel officials.\n\nThe Post had previously reported that duri

In [12]:
dataset.shape

(1699, 5)

In [13]:
dataset['narratives']

0       [Blaming the war on others rather than the inv...
1       [Discrediting the West, Diplomacy, Discreditin...
2                                [Distrust towards Media]
3            [Discrediting Ukraine, Discrediting Ukraine]
4                                      [Praise of Russia]
                              ...                        
1694                       [Amplifying war-related fears]
1695    [Criticism of climate movement, Downplaying cl...
1696    [Criticism of institutions and authorities, Co...
1697                           [Speculating war outcomes]
1698                           [Amplifying Climate Fears]
Name: narratives, Length: 1699, dtype: object

In [14]:
unique_narratives = dataset['narratives'].explode().unique()
unique_narratives

array(['Blaming the war on others rather than the invader',
       'Discrediting the West, Diplomacy',
       'Hidden plots by secret schemes of powerful groups',
       'Discrediting Ukraine', 'Praise of Russia',
       'Distrust towards Media', 'Russia is the Victim',
       'Negative Consequences for the West', 'Speculating war outcomes',
       'Amplifying war-related fears', 'Overpraising the West',
       'Downplaying climate change',
       'Criticism of institutions and authorities',
       'Questioning the measurements and science',
       'Climate change is beneficial', 'Criticism of climate policies',
       'Criticism of climate movement', 'Amplifying Climate Fears',
       'Other', 'Controversy about green technologies',
       'Green policies are geopolitical instruments'], dtype=object)

In [15]:
print(len(dataset['narratives'].explode().value_counts()))
dataset['narratives'].explode().value_counts()

21


narratives
Discrediting Ukraine                                 584
Discrediting the West, Diplomacy                     452
Praise of Russia                                     406
Amplifying Climate Fears                             357
Other                                                324
Amplifying war-related fears                         297
Russia is the Victim                                 229
Criticism of institutions and authorities            216
Blaming the war on others rather than the invader    194
Speculating war outcomes                             132
Criticism of climate policies                        127
Negative Consequences for the West                   104
Criticism of climate movement                         84
Hidden plots by secret schemes of powerful groups     84
Downplaying climate change                            68
Distrust towards Media                                53
Overpraising the West                                 51
Controversy about gr

In [16]:
unique_subnarratives = dataset['subnarratives'].explode().unique()
unique_subnarratives

array(['The West are the aggressors', 'Other', 'The West is weak',
       'Ukraine is a puppet of the West',
       'Ukraine is associated with nazism',
       'Russia is a guarantor of peace and prosperity',
       'The West does not care about Ukraine, only about its interests',
       'The EU is divided',
       'Western media is an instrument of propaganda',
       'Discrediting Ukrainian government and officials and policies',
       'The West is overreacting', 'UA is anti-RU extremists',
       'Discrediting Ukrainian nation and society',
       'Discrediting Ukrainian military',
       'Ukrainian media cannot be trusted',
       'Praise of Russian military might', 'The West is russophobic',
       'Ukrainian army is collapsing',
       'Russia has international support from a number of countries and people',
       'Praise of Russian President Vladimir Putin',
       'By continuing the war we risk WWIII', 'Ukraine is the aggressor',
       'Russia actions in Ukraine are only sel

In [17]:
len(unique_subnarratives)

74

In [18]:
pd.set_option('display.max_rows', 100)

dataset['subnarratives'].explode().value_counts()

subnarratives
Other                                                                     1164
Amplifying existing fears of global warming                                178
Discrediting Ukrainian government and officials and policies               157
Praise of Russian military might                                           145
The West are the aggressors                                                112
Ukraine is a puppet of the West                                            106
Russia is a guarantor of peace and prosperity                              101
Discrediting Ukrainian military                                            100
There is a real possibility that nuclear weapons will be employed           96
Criticism of national governments                                           85
The West does not care about Ukraine, only about its interests              85
Ukraine is the aggressor                                                    78
Russia has international support from 

### 1.2 Encoding classification labels

In [19]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb_narratives = MultiLabelBinarizer()
mlb_subnarratives = MultiLabelBinarizer()

In [20]:
narratives_binary = mlb_narratives.fit_transform(dataset['narratives'])
subnarratives_binary = mlb_subnarratives.fit_transform(dataset['subnarratives'])

dataset['narratives_encoded'] = narratives_binary.tolist()
dataset['subnarratives_encoded'] = subnarratives_binary.tolist()

In [21]:
dataset.head()

,language,article_id,content,narratives,subnarratives,narratives_encoded,subnarratives_encoded
0,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,[Blaming the war on others rather than the inv...,"[The West are the aggressors, Other, The West ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,RU,RU-URW-1175.txt,В ЕС испугались последствий популярности правы...,"[Discrediting the West, Diplomacy, Discreditin...","[The West is weak, Other, The EU is divided]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,RU,RU-URW-1149.txt,Возможность признания Аллы Пугачевой иностранн...,[Distrust towards Media],[Western media is an instrument of propaganda],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,RU,RU-URW-1015.txt,Азаров рассказал о смене риторики Киева по пер...,"[Discrediting Ukraine, Discrediting Ukraine]","[Ukraine is a puppet of the West, Discrediting...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,RU,RU-URW-1001.txt,В россиянах проснулась массовая любовь к путеш...,[Praise of Russia],[Russia is a guarantor of peace and prosperity],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [22]:
dataset['narratives'].explode().value_counts()

narratives
Discrediting Ukraine                                 584
Discrediting the West, Diplomacy                     452
Praise of Russia                                     406
Amplifying Climate Fears                             357
Other                                                324
Amplifying war-related fears                         297
Russia is the Victim                                 229
Criticism of institutions and authorities            216
Blaming the war on others rather than the invader    194
Speculating war outcomes                             132
Criticism of climate policies                        127
Negative Consequences for the West                   104
Criticism of climate movement                         84
Hidden plots by secret schemes of powerful groups     84
Downplaying climate change                            68
Distrust towards Media                                53
Overpraising the West                                 51
Controversy about gr

In [23]:
subnarratives_counts = dataset['subnarratives'].explode().value_counts()
print(len(subnarratives_counts))
subnarratives_counts

74


subnarratives
Other                                                                     1164
Amplifying existing fears of global warming                                178
Discrediting Ukrainian government and officials and policies               157
Praise of Russian military might                                           145
The West are the aggressors                                                112
Ukraine is a puppet of the West                                            106
Russia is a guarantor of peace and prosperity                              101
Discrediting Ukrainian military                                            100
There is a real possibility that nuclear weapons will be employed           96
Criticism of national governments                                           85
The West does not care about Ukraine, only about its interests              85
Ukraine is the aggressor                                                    78
Russia has international support from 

### 1.3 Cleaning articles

In [24]:
language_model_map = {
    "BG": "xx_ent_wiki_sm",
    "EN": "en_core_web_sm",
    "HI": "xx_ent_wiki_sm",
    "PT": "pt_core_news_sm",
    "RU": "ru_core_news_sm",
}

!python3 -m spacy download xx_ent_wiki_sm
!python3 -m spacy download pt_core_news_sm
!python3 -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 10.5 MB/s eta 0:00:00 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 10.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 10.2 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [25]:
!pip3 -q install emoji

In [26]:
import spacy
import emoji

nlp_models = {lang: spacy.load(model) for lang, model in language_model_map.items()}

In [27]:
import re

def clean_article_with_paragraphs(article_text, language_code):
    nlp = nlp_models.get(language_code, nlp_models["EN"])

    parts = re.split(r'\n{2,}', article_text)
    if len(parts) > 2:
        header = parts[0].strip()
        footer = parts[-1].strip()
        body = parts[1:-1]
    else:
        header = parts[0].strip() if len(parts) > 0 else ""
        footer = parts[1].strip() if len(parts) > 1 else ""
        body = []

    def clean_paragraph(paragraph):
        paragraph = re.sub(
            r'http\S+|www\S+|https\S+|[a-zA-Z0-9.-]+\.com|[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+|@[A-Za-z0-9_]+',
            '',
            paragraph
        )
        doc = nlp(paragraph)
        cleaned_tokens = []

        important_entity_types = ["PERSON", "ORG", "GPE"]

        for token in doc:
            if token.is_space or emoji.is_emoji(token.text):
                continue

            if token.ent_type_ in important_entity_types:
                cleaned_tokens.append(token.text + token.whitespace_)
            else:
                cleaned_tokens.append(token.text.lower() + token.whitespace_)

        return "".join(cleaned_tokens).strip()

    cleaned_header = f"<PARA>{clean_paragraph(header)}</PARA>" if header else ""
    cleaned_footer = f"<PARA>{clean_paragraph(footer)}</PARA>" if footer else ""

    cleaned_body = " ".join([clean_paragraph(paragraph) for paragraph in body])

    combined_text = "\n\n".join(filter(None, [cleaned_header, cleaned_body, cleaned_footer]))
    return combined_text.strip()

dataset["content"] = dataset.apply(
    lambda row: clean_article_with_paragraphs(row["content"], row["language"]),
    axis=1
)

In [35]:
row = 6
english_article = dataset[dataset['language'] == 'EN'].iloc[row].content
english_article

'<PARA>Shapps Says uk facing ‘pre-war world’ and can’t risk cutting defence spending</PARA>\n\nDefence secretary Grant Shapps has said Britain is in a “pre-war world” and cannot afford to reduce spending on the military. in a major speech at Lancaster House in London, mr. Shapps said the “peace dividend” that followed the end of the cold war in the early 1990s was over. mr. Shapps said: “because the era of the peace dividend is over, in five years’ time we could be looking at multiple theatres including Russia, China, Iran, and North Korea.” “ask yourself, looking at today’s conflicts across the world, is it more likely that that number grows or reduces? i suspect we all know the answer. it’s likely to grow,” he added. mr. Shapps also announced 20,000 british service personnel would take part in NATO’s largest military exercises since the cold war, following the announcement last week of a £2.5 billion support package to Ukraine. over the weekend the RAF carried out air strikes with U.

In [36]:
def split_into_sections(content):
    parts = re.split(r'<PARA>|</PARA>', content)
    parts = [p.strip() for p in parts if p.strip()]

    if len(parts) == 1:
        return parts[0], "", ""
    elif len(parts) == 2:
        return parts[0], parts[1], ""
    else:
        header = parts[0]
        footer = parts[-1]
        body = " ".join(parts[1:-1])
        return header, body, footer

In [37]:
header, body, footer = split_into_sections(english_article)
print("Header: ", header)
print("\n\n")
print("Body: ", body)
print("\n\n")
print("Footer: ", footer)

Header:  Shapps Says uk facing ‘pre-war world’ and can’t risk cutting defence spending



Body:  Defence secretary Grant Shapps has said Britain is in a “pre-war world” and cannot afford to reduce spending on the military. in a major speech at Lancaster House in London, mr. Shapps said the “peace dividend” that followed the end of the cold war in the early 1990s was over. mr. Shapps said: “because the era of the peace dividend is over, in five years’ time we could be looking at multiple theatres including Russia, China, Iran, and North Korea.” “ask yourself, looking at today’s conflicts across the world, is it more likely that that number grows or reduces? i suspect we all know the answer. it’s likely to grow,” he added. mr. Shapps also announced 20,000 british service personnel would take part in NATO’s largest military exercises since the cold war, following the announcement last week of a £2.5 billion support package to Ukraine. over the weekend the RAF carried out air strikes with 

In [38]:
shuffled_dataset = dataset.sample(frac = 1)
max_rows = 25
for i, row in enumerate(shuffled_dataset.iterrows()):
    content = shuffled_dataset.iloc[i].content
    header, body, footer = split_into_sections(content)
    print("Header: ", header)
    print("\n\n")
    print("Body: ", body)
    print("\n\n")
    print("Footer: ", footer)
    max_rows -= 1
    if max_rows == 0:
        break

Header:  viktor orban in china: हंगरी के प्रधानमंत्री विक्टर ओरबान रूस यात्रा की यूक्रेन और यूरोपीय संघ ने रूस की यात्रा को लेकर ओरबान की आलोचना की थी.Trending Photosrussia ukraine war: लहंगरी के प्रधानमंत्री विक्टर ओरबान रूस और यूक्रेन की यात्रा के बाद सोमवार को अचानक चीन पहुंचे. वह यूक्रेन में शांति समझौते की संभावनाओं पर चर्चा करने के लिए चीन पहुंचे हैं. सरकारी ब्रॉडकास्टर सीसीटीवी ने जानकारी दी कि उन्होंने राष्ट्रपति शी चिनपिंग से मुलाकात की.ओरबान की यह यात्रा पिछले सप्ताह यूक्रेन और रूस की इसी तरह की अघोषित यात्रा के कुछ ही दिनों बाद हो रही है. हालांकि यूक्रेन और यूरोपीय संघ ने रूस की यात्रा को लेकर ओरबान की आलोचना की थी.हंगरी किसी से भी बात कर सकता हैओरबान ने कहा, ‘दोनों युद्धरत देशों से बात करने वाले देशों की संख्या कम होती जा रही है. हंगरी धीरे-धीरे यूरोप में इकलौता ऐसा देश बनकर उभर रहा है, जो हर किसी से बात कर सकता है.’आरेबान की रूस यात्रा पर विवादहंगरी ने महीने की शुरुआत में यूरोपीय संघ की छह महीने की क्रमिक अध्यक्षता संभाली थी. इस दौरान रूस के राष्ट्रपति व्लादिमीर पुतिन ने स

In [39]:
!pip install -q iterative-stratification

In [40]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

def stratified_train_val_split(data, labels_column, train_size=0.8, splits=5, shuffle=True, min_instances=2):
    if shuffle:
        shuffled_indices = np.arange(len(data))
        np.random.shuffle(shuffled_indices)
        data = data.iloc[shuffled_indices].reset_index(drop=True)

    labels = np.array(data[labels_column].tolist())
    rare_indices = []
    common_indices = []

    class_counts = labels.sum(axis=0)
    rare_classes = np.where(class_counts <= min_instances)[0]

    for idx, label_row in enumerate(labels):
        if any(label_row[rare_classes]):
            rare_indices.append(idx)
        else:
            common_indices.append(idx)

    rare_data = data.iloc[rare_indices]
    rare_labels = labels[rare_indices]
    train_rare = rare_data.iloc[:len(rare_data) // 2].reset_index(drop=True)
    val_rare = rare_data.iloc[len(rare_data) // 2:].reset_index(drop=True)

    common_data = data.iloc[common_indices].reset_index(drop=True)
    common_labels = labels[common_indices]

    mskf = MultilabelStratifiedKFold(n_splits=splits)
    for train_idx, val_idx in mskf.split(np.zeros(len(common_labels)), common_labels):
        train_common = common_data.iloc[train_idx]
        val_common = common_data.iloc[val_idx]
        break

    train_data = pd.concat([train_rare, train_common]).reset_index(drop=True)
    val_data = pd.concat([val_rare, val_common]).reset_index(drop=True)

    return train_data, val_data

(dataset_train), (dataset_val) = stratified_train_val_split(
    dataset,
    labels_column="subnarratives_encoded",
    min_instances=2
)

In [41]:
train_sub_nar_counts = dataset_train['subnarratives'].explode().value_counts()
print(len(train_sub_nar_counts))
train_sub_nar_counts

73


subnarratives
Other                                                                     933
Amplifying existing fears of global warming                               142
Discrediting Ukrainian government and officials and policies              126
Praise of Russian military might                                          116
The West are the aggressors                                                89
Ukraine is a puppet of the West                                            85
Russia is a guarantor of peace and prosperity                              82
Discrediting Ukrainian military                                            80
There is a real possibility that nuclear weapons will be employed          77
Criticism of national governments                                          68
The West does not care about Ukraine, only about its interests             68
Ukraine is the aggressor                                                   62
Russia has international support from a number of 

In [42]:
val_sub_nar_counts = dataset_val['subnarratives'].explode().value_counts()
print(len(val_sub_nar_counts))
val_sub_nar_counts

72


subnarratives
Other                                                                     231
Amplifying existing fears of global warming                                36
Discrediting Ukrainian government and officials and policies               31
Praise of Russian military might                                           29
The West are the aggressors                                                23
Ukraine is a puppet of the West                                            21
Discrediting Ukrainian military                                            20
There is a real possibility that nuclear weapons will be employed          19
Russia is a guarantor of peace and prosperity                              19
Criticism of national governments                                          17
The West does not care about Ukraine, only about its interests             17
Ukraine is the aggressor                                                   16
Russia has international support from a number of 

### Data augmentantion

In [43]:
lower_freq_bound = 20
low_freq_narratives = dataset_train['narratives'].explode().value_counts()
low_freq_narratives = low_freq_narratives[low_freq_narratives < lower_freq_bound].index.tolist()

low_freq_subnarratives = dataset_train['subnarratives'].explode().value_counts()
low_freq_subnarratives = low_freq_subnarratives[low_freq_subnarratives < lower_freq_bound].index.tolist()

In [40]:
low_freq_narratives

['Climate change is beneficial', 'Green policies are geopolitical instruments']

In [41]:
low_freq_subnarratives

['Earth will be uninhabitable soon',
 'West is tired of Ukraine',
 'Ad hominem attacks on key activists',
 'The West belongs in the right side of history',
 'Climate policies are only for profit',
 'Blaming global elites',
 'Russian army is collapsing',
 'UA is anti-RU extremists',
 'The West has the strongest international support',
 'Scientific community is unreliable',
 'Climate cycles are natural',
 'Renewable energy is unreliable',
 'Renewable energy is costly',
 'Methodologies/metrics used are unreliable/faulty',
 'Climate movement is corrupt',
 'Discrediting Ukrainian nation and society',
 'Criticism of the EU',
 'The West is overreacting',
 'Whatever we do it is already too late',
 'The conflict will increase the Ukrainian refugee flows to Europe',
 'NATO will destroy Russia',
 'Rewriting Ukraine’s history',
 'Human activities do not impact climate change',
 'Temperature increase is beneficial',
 'Russian invasion has strong national support',
 'Russian army will lose all the o

In [42]:
low_freq_data = dataset_train[
    dataset_train['narratives'].apply(lambda x: any(label in low_freq_narratives for label in x)) |
    dataset_train['subnarratives'].apply(lambda x: any(label in low_freq_subnarratives for label in x))
]

In [43]:
print(low_freq_data.shape)
low_freq_data.head()

(218, 7)


,language,article_id,content,narratives,subnarratives,narratives_encoded,subnarratives_encoded
0,EN,EN_CC_200022.txt,<PARA>Denmark to Punish Farmers for cow ‘emiss...,"[Criticism of institutions and authorities, Cr...","[Criticism of national governments, Other, Met...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
1,PT,PT_CC_416.txt,<PARA>da patranha do aquecimento global</PARA>...,"[Questioning the measurements and science, Hid...","[Scientific community is unreliable, Climate a...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
17,HI,HI_130.txt,<PARA>जलवायु परिवर्तन नीति - स्नोहोमिश काउंटी ...,"[Amplifying Climate Fears, Criticism of climat...","[Amplifying existing fears of global warming, ...","[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
23,BG,BG_1249.txt,<PARA>нищо не може да спре катастрофата: на та...,"[Amplifying Climate Fears, Amplifying Climate ...","[Earth will be uninhabitable soon, Doomsday sc...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
25,EN,EN_CC_300005.txt,<PARA>NY’s electric school bus mandate won’t m...,"[Criticism of climate policies, Criticism of c...","[Climate policies are ineffective, Climate pol...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."


In [44]:
low_freq_data['language'].value_counts()

language
EN    84
BG    53
PT    39
HI    33
RU     9
Name: count, dtype: int64

In [45]:
!pip install -q googletrans==4.0.0-rc1

In [46]:
from googletrans import Translator
from time import sleep

translator = Translator()

language_map = {
    'EN': 'en',  # English
    'BG': 'bg',  # Bulgarian
    'PT': 'pt',  # Portuguese
    'HI': 'hi',  # Hindi
}
# TODO: Change this to use a model instead
def translate_text_google(text, target_lang='en'):
    try:
        # sleeping to avoid rate-limiting
        sleep(1)
        translated = translator.translate(text, dest=target_lang)
        return translated.text
    except Exception as e:
        print(f"Translation failed for language {target_lang} with error: {e}")
        return ""

text = "Hola, ¿cómo estás?"
translated_text = translate_text_google(text)
print(translated_text)

Hello how are you?


In [47]:
# TODO: Maybe translate only an arbitrary language, not back to it's origin.
def augment_with_translation(text, lang='EN'):
    try:
        if lang != 'EN':
            # translate to English, then back to original language
            translated_text = translate_text_google(text, target_lang='en')
            return translate_text_google(translated_text, target_lang=language_map.get(lang, 'EN'))
        else:
            # translate to FR, then back to EN
            temp_translation = translate_text_google(text, target_lang='fr')
            return translate_text_google(temp_translation, target_lang='en')
    except Exception as e:
        print(f"Error processing text: {e}")
        return text

In [48]:
data_augment = False
augmented_df = None

def translate_and_augment_data(low_freq_data):
    augmented_data = []
    low_freq_data = low_freq_data.reset_index(drop=True)

    for index, row in low_freq_data.iterrows():
        original_text = row['content']
        language = row['language']
        article_id = row['article_id']
        narratives = row['narratives']
        subnarratives = row['subnarratives']
        narratives_encoded = row['narratives_encoded']
        subnarratives_encoded = row['subnarratives_encoded']

        try:
            if language != 'EN':
                translated_text = augment_with_translation(original_text, lang=language)
            else:
                translated_text = augment_with_translation(original_text, lang='EN')

            augmented_data.append({
                'language': language,
                'article_id': article_id,
                'content': translated_text,
                'narratives': narratives,
                'subnarratives': subnarratives,
                'narratives_encoded': narratives_encoded,
                'subnarratives_encoded': subnarratives_encoded
            })
        except Exception as e:
            print(f"Error processing row {index}: {e}")

    return pd.DataFrame(augmented_data)

if data_augment: augmented_df = translate_and_augment_data(low_freq_data)

In [ ]:
blank_content = augmented_df[augmented_df['content'].str.strip() == '']
len(blank_content)

In [ ]:
augmented_df = augmented_df[augmented_df['content'].str.strip() != '']

In [ ]:
print(len(augmented_df))
augmented_df.head()

In [ ]:
augmented_df['subnarratives'].explode().value_counts()

In [ ]:
print(augmented_df['narratives_encoded'].head())

In [ ]:
print(augmented_df['subnarratives_encoded'].head())

In [ ]:
dataset_train = pd.concat([dataset_train, augmented_df], ignore_index=True)

In [ ]:
dataset_train.shape

In [ ]:
dataset_train['narratives'].explode().value_counts()

### 1.4 Getting embeddings for the articles

In [44]:
embeddings_dir = '../data/embeddings/narrative_classification_multilingual/'
embedding_file_name = 'all_embeddings.npy'
embeddings_full_path = embeddings_dir + embedding_file_name

In [45]:
import os

def are_embeddings_saved(filepath):
    if os.path.exists(filepath):
        return True
    return False

In [46]:
are_embeddings_saved(embeddings_full_path)

False

In [47]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

labse_model = SentenceTransformer('sentence-transformers/LaBSE')
labse_tokenizer = AutoTokenizer.from_pretrained(
    'sentence-transformers/LaBSE'
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the 

In [48]:
texts = [
    "This is a news article about politics.",  # English
    "यह राजनीति के बारे में एक समाचार लेख है।",  # Hindi
    "Este é um artigo de notícias sobre política.",  # Portuguese
    "Това е новинарска статия за политика.",  # Bulgarian
    "The weather was nice today."  # Unrelated English sentence
]

embeddings = labse_model.encode(texts)

In [49]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim_matrix = cosine_similarity(embeddings)

print("Cosine Similarity Matrix:")
print(cos_sim_matrix)

Cosine Similarity Matrix:
[[1.         0.8829176  0.9265815  0.8651781  0.24318342]
 [0.8829176  1.0000002  0.96578115 0.89237857 0.39229706]
 [0.9265815  0.96578115 1.0000002  0.9107212  0.36290163]
 [0.8651781  0.89237857 0.9107212  1.0000002  0.2976809 ]
 [0.24318342 0.39229706 0.36290163 0.2976809  1.        ]]


In [50]:
import torch 
class EmbeddingProcessor:
    def __init__(self, model, tokenizer, max_length=512):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length

    def _check_truncation(self, content):
        tokens = self.tokenizer(content, return_tensors="pt", truncation=False)
        num_tokens = len(tokens['input_ids'][0])
        max_seq_len = self.tokenizer.model_max_length
        
        if num_tokens > max_seq_len:
            print(f"Truncation warning: Article has {num_tokens} tokens, exceeding the limit of {max_seq_len}.")
            return True
        return False

    def _split_long_paragraph_and_get_embeddings(self, body, max_length=512, strategy="sum"):
        chunks = []
        embeddings = []
        while len(body) > max_length:
            split_index = body.rfind(' ', 0, max_length)
    
            if split_index == -1:
                split_index = max_length
    
            chunk = body[:split_index].strip()
            chunks.append(chunk)
            embeddings.append(torch.tensor(self.model.encode(chunk)))
    
            body = body[split_index:].strip()
    
        if body:
            chunks.append(body.strip())
            embeddings.append(torch.tensor(self.model.encode(body.strip())))
    
        aggregated_embedding = self._aggregate_embeddings(embeddings, strategy=strategy)
        return aggregated_embedding, chunks


    def _aggregate_embeddings(self, paragraph_embeddings, strategy="sum"):
        if strategy == "mean":
            return torch.mean(torch.stack(paragraph_embeddings), dim=0).numpy()
        elif strategy == "sum":
            return torch.sum(torch.stack(paragraph_embeddings), dim=0).numpy()
        elif strategy == "concat":
            return torch.cat(paragraph_embeddings, dim=0).numpy()
        elif strategy == "rms":
            return torch.sqrt(torch.mean(torch.stack([torch.square(e) for e in paragraph_embeddings]), dim=0)).numpy()
        else:
            raise ValueError(f"Unsupported aggregation strategy: {strategy}")


    def get_embeddings(self, content, strategy="mean"):
        embeddings = []
    
        header, body, footer = split_into_sections(content)
        if header:
            header_embedding = torch.tensor(self.model.encode(header))
            embeddings.append(header_embedding)
    
        if body:
            body_embeddings, body_chunks = self._split_long_paragraph_and_get_embeddings(body, strategy="sum")
            body_embeddings = torch.tensor(body_embeddings, dtype=torch.float32)
            embeddings.append(body_embeddings)
    
        if footer:
            footer_embedding = torch.tensor(self.model.encode(footer))
            embeddings.append(footer_embedding)
    
        aggregated_embeddings = self._aggregate_embeddings(embeddings, strategy)
        return aggregated_embeddings

In [51]:
processor = EmbeddingProcessor(model=labse_model, tokenizer=labse_tokenizer)

In [52]:
train_embeddings = dataset_train['content'].apply(
    lambda content: processor.get_embeddings(content, strategy="sum")
)

In [53]:
train_embeddings = np.array(train_embeddings.tolist())

In [54]:
train_embeddings.shape

(1367, 768)

In [55]:
val_embeddings = dataset_val['content'].apply(
    lambda content: processor.get_embeddings(content, strategy="sum")
)

In [56]:
val_embeddings = np.array(val_embeddings.tolist())
val_embeddings.shape

(332, 768)

In [57]:
y_train_nar = dataset_train['narratives_encoded'].tolist()
y_val_nar = dataset_val['narratives_encoded'].tolist()

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

ovr_logistic_nar = OneVsRestClassifier(LogisticRegression(max_iter=1000, class_weight='balanced'))

In [59]:
train_embeddings.shape

(1367, 768)

In [60]:
ovr_logistic_nar.fit(train_embeddings, y_train_nar)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                 max_iter=1000))

In [61]:
import warnings
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

def get_classification_report(y_true, y_pred):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        report = classification_report(y_true, y_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    return report_df

def get_cross_val_score(model, x, y, scoring='f1_macro', splits=3):
    """Perform cross-validation and compute scores."""
    cv = StratifiedKFold(n_splits=splits, shuffle=True)
    cross_val_scores = cross_val_score(model, x, y, cv=cv, scoring=scoring)
    print(f"Cross-validation scores: {cross_val_scores}")
    print(f"Mean CV F1 Score: {cross_val_scores.mean()}")

In [62]:
import warnings
from sklearn.metrics import (
    hamming_loss,
)

def evaluate_model(model, x, y_true):
    y_pred = model.predict(x)

    classification_report_df = get_classification_report(y_true, y_pred)
    print("Classification Report:")
    print(classification_report_df)
    print("\n")

    hamming = hamming_loss(y_true, y_pred)
    print(f"Hamming Loss: {hamming:.4f}")
    print("\n")

In [63]:
evaluate_model(ovr_logistic_nar, val_embeddings, y_val_nar)

Classification Report:
              precision    recall  f1-score  support
0              0.758621  0.977778  0.854369     45.0
1              0.521739  0.750000  0.615385     48.0
2              0.360000  0.729730  0.482143     37.0
3              0.400000  0.500000  0.444444      4.0
4              0.294118  0.833333  0.434783      6.0
5              0.388889  0.466667  0.424242     15.0
6              0.343750  0.647059  0.448980     17.0
7              0.531915  0.781250  0.632911     32.0
8              0.517544  0.776316  0.621053     76.0
9              0.411765  0.711864  0.521739     59.0
10             0.166667  0.333333  0.222222      9.0
11             0.333333  0.600000  0.428571     10.0
12             0.333333  0.500000  0.400000      2.0
13             0.200000  0.500000  0.285714     14.0
14             0.214286  0.428571  0.285714     21.0
15             0.425532  0.606061  0.500000     66.0
16             0.080000  0.222222  0.117647      9.0
17             0.436893

In [64]:
y_train_sub_nar = dataset_train['subnarratives_encoded'].tolist()
y_val_sub_nar = dataset_val['subnarratives_encoded'].tolist()

In [65]:
ovr_logistic_sub = OneVsRestClassifier(LogisticRegression(max_iter=1000, class_weight='balanced'))
ovr_logistic_sub.fit(train_embeddings, y_train_sub_nar)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 49 is present in all training examples.
  warnings.warn(


OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                 max_iter=1000))

In [66]:
evaluate_model(ovr_logistic_sub, val_embeddings, y_val_sub_nar)

Classification Report:
              precision    recall  f1-score  support
0              0.142857  0.250000  0.181818      4.0
1              0.727273  0.888889  0.800000     36.0
2              0.600000  0.600000  0.600000      5.0
3              0.200000  0.538462  0.291667     13.0
4              0.000000  0.000000  0.000000      2.0
5              0.333333  0.500000  0.400000      2.0
6              0.250000  0.285714  0.266667      7.0
7              0.250000  0.333333  0.285714      3.0
8              0.300000  0.428571  0.352941      7.0
9              0.250000  0.333333  0.285714      3.0
10             0.117647  0.285714  0.166667      7.0
11             0.250000  0.500000  0.333333      4.0
12             0.235294  0.444444  0.307692      9.0
13             0.000000  0.000000  0.000000      1.0
14             0.277778  0.833333  0.416667      6.0
15             0.344828  0.588235  0.434783     17.0
16             0.392857  0.733333  0.511628     15.0
17             0.000000

In [67]:
import joblib

joblib.dump(ovr_logistic_nar, './ovr_logistic_narrative_model.pkl')
joblib.dump(ovr_logistic_sub, './ovr_logistic_subnarrative_model.pkl')

print("Models saved successfully.")

Models saved successfully.


In [68]:
import torch
import torch.nn as nn

class MultiTaskClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_narratives, num_subnarratives, dropout_rate=0.3):
        super(MultiTaskClassifier, self).__init__()
        
        self.shared_layer = nn.Sequential(
            nn.Linear(input_size, hidden_size * 2),
            nn.BatchNorm1d(hidden_size * 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        
        self.narrative_head = nn.Sequential(
            nn.Linear(hidden_size * 2, num_narratives),
            nn.Sigmoid()
        )
        
        self.subnarrative_head = nn.Sequential(
            nn.Linear(hidden_size * 2, num_subnarratives),
            nn.Sigmoid()
        )

    def forward(self, x):
        shared_output = self.shared_layer(x)
        narratives = self.narrative_head(shared_output)
        subnarratives = self.subnarrative_head(shared_output)
        return narratives, subnarratives


train_embeddings_tensor = torch.tensor(train_embeddings, dtype=torch.float32)
val_embeddings_tensor = torch.tensor(val_embeddings, dtype=torch.float32)
model = MultiTaskClassifier(input_size=768, hidden_size=128, num_narratives=len(mlb_narratives.classes_), num_subnarratives=len(mlb_subnarratives.classes_))
narratives, subnarratives = model(train_embeddings_tensor)
print(narratives.shape, subnarratives.shape)

torch.Size([1367, 21]) torch.Size([1367, 74])


In [69]:
y_train_nar = torch.tensor(y_train_nar, dtype=torch.float32)
y_train_sub_nar = torch.tensor(y_train_sub_nar, dtype=torch.float32)

y_val_nar = torch.tensor(y_val_nar, dtype=torch.float32)
y_val_sub_nar = torch.tensor(y_val_sub_nar, dtype=torch.float32)

In [70]:
import torch
import torch.nn as nn

def compute_class_weights(y_train):
    total_samples = y_train.shape[0]
    class_weights = []
    for label in range(y_train.shape[1]):
        pos_count = y_train[:, label].sum().item()
        neg_count = total_samples - pos_count
        pos_weight = total_samples / (2 * pos_count) if pos_count > 0 else 0
        neg_weight = total_samples / (2 * neg_count) if neg_count > 0 else 0
        class_weights.append((pos_weight, neg_weight))
    return class_weights

class WeightedBCELoss(nn.Module):
    def __init__(self, class_weights):
        super().__init__()
        self.class_weights = class_weights

    def forward(self, probs, targets):
        bce_loss = 0
        epsilon = 1e-7
        for i, (pos_weight, neg_weight) in enumerate(self.class_weights):
            prob = probs[:, i] 
            bce = -pos_weight * targets[:, i] * torch.log(prob + epsilon) - \
                  neg_weight * (1 - targets[:, i]) * torch.log(1 - prob + epsilon)
            bce_loss += bce.mean()
        return bce_loss / len(self.class_weights)

class_weights_nar = compute_class_weights(y_train_nar) 
narrative_criterion = WeightedBCELoss(class_weights_nar)

In [71]:
class_weights_sub_nar = compute_class_weights(y_train_sub_nar) 
subnarrative_criterion = WeightedBCELoss(class_weights_sub_nar)

In [72]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [73]:
def train_with_early_stopping(
    model,
    train_embeddings,
    y_train_nar,
    y_train_sub_nar,
    val_embeddings,
    y_val_nar,
    y_val_sub_nar,
    optimizer,
    narrative_criterion,
    subnarrative_criterion,
    patience=3,
    num_epochs=100,
):
    best_val_loss = float('inf')
    best_model = None
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        narratives, subnarratives = model(train_embeddings)

        narrative_loss = narrative_criterion(narratives, y_train_nar)
        subnarrative_loss = subnarrative_criterion(subnarratives, y_train_sub_nar)
        loss = narrative_loss + subnarrative_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_narratives, val_subnarratives = model(val_embeddings)
            val_narrative_loss = narrative_criterion(val_narratives, y_val_nar)
            val_subnarrative_loss = subnarrative_criterion(val_subnarratives, y_val_sub_nar)
            val_loss = val_narrative_loss + val_subnarrative_loss

        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Training Loss: {loss.item():.4f} "
              f"(Narrative: {narrative_loss.item():.4f}, Subnarrative: {subnarrative_loss.item():.4f}), "
              f"Validation Loss: {val_loss.item():.4f} "
              f"(Narrative: {val_narrative_loss.item():.4f}, Subnarrative: {val_subnarrative_loss.item():.4f})")

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            patience_counter = 0
            best_model = model.state_dict()
        else:
            patience_counter += 1
            print(f"Validation loss did not improve for {patience_counter} epoch(s).")

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

    if best_model:
        model.load_state_dict(best_model)

    return model

In [74]:
trained_model = train_with_early_stopping(
    model=model,
    train_embeddings=train_embeddings_tensor,
    y_train_nar=y_train_nar,
    y_train_sub_nar=y_train_sub_nar,
    val_embeddings=val_embeddings_tensor,
    y_val_nar=y_val_nar,
    y_val_sub_nar=y_val_sub_nar,
    optimizer=optimizer,
    narrative_criterion=narrative_criterion,
    subnarrative_criterion=subnarrative_criterion,
    patience=3
)

Epoch 1/100, Training Loss: 1.4255 (Narrative: 0.7155, Subnarrative: 0.7100), Validation Loss: 1.4718 (Narrative: 0.7296, Subnarrative: 0.7422)
Epoch 2/100, Training Loss: 1.2901 (Narrative: 0.6360, Subnarrative: 0.6540), Validation Loss: 1.4600 (Narrative: 0.7213, Subnarrative: 0.7387)
Epoch 3/100, Training Loss: 1.2029 (Narrative: 0.5887, Subnarrative: 0.6141), Validation Loss: 1.4470 (Narrative: 0.7128, Subnarrative: 0.7342)
Epoch 4/100, Training Loss: 1.1434 (Narrative: 0.5567, Subnarrative: 0.5867), Validation Loss: 1.4328 (Narrative: 0.7041, Subnarrative: 0.7287)
Epoch 5/100, Training Loss: 1.0956 (Narrative: 0.5331, Subnarrative: 0.5625), Validation Loss: 1.4174 (Narrative: 0.6953, Subnarrative: 0.7221)
Epoch 6/100, Training Loss: 1.0545 (Narrative: 0.5141, Subnarrative: 0.5404), Validation Loss: 1.4011 (Narrative: 0.6864, Subnarrative: 0.7147)
Epoch 7/100, Training Loss: 1.0208 (Narrative: 0.4978, Subnarrative: 0.5230), Validation Loss: 1.3843 (Narrative: 0.6776, Subnarrative: 

In [75]:
from sklearn.metrics import classification_report, f1_score

def evaluate_model(
    model,
    embeddings,
    y_nar_true,
    y_sub_nar_true,
    thresholds=np.arange(0.1, 1.0, 0.1),
    target_names_nar=None,
    target_names_sub=None
):
    best_threshold = 0
    best_f1 = 0
    best_classification_report_nar = None
    best_classification_report_sub = None

    for threshold in thresholds:
        with torch.no_grad():
            nar_pred_logits, sub_nar_pred_logits = model(embeddings)
            
            nar_predictions = (nar_pred_logits >= threshold).int().cpu().numpy()
            sub_nar_predictions = (sub_nar_pred_logits >= threshold).int().cpu().numpy()
            
            y_nar_true_np = y_nar_true.cpu().numpy()
            y_sub_nar_true_np = y_sub_nar_true.cpu().numpy()
            
            classification_rep_nar = classification_report(
                y_nar_true_np, nar_predictions, target_names=target_names_nar, zero_division=0
            )
            classification_rep_sub = classification_report(
                y_sub_nar_true_np, sub_nar_predictions, target_names=target_names_sub, zero_division=0
            )
            f1_nar = f1_score(y_nar_true_np, nar_predictions, average='macro')
            f1_sub = f1_score(y_sub_nar_true_np, sub_nar_predictions, average='macro')
            
            avg_f1 = (f1_nar + f1_sub) / 2
            
            if avg_f1 > best_f1:
                best_f1 = avg_f1
                best_threshold = threshold
                best_classification_report_nar = classification_rep_nar
                best_classification_report_sub = classification_rep_sub

    print(f"Best Threshold: {best_threshold}, Best F1 Score: {best_f1}")
    print("\nBest Narratives Classification Report:")
    print(best_classification_report_nar)
    print("\nBest Sub-Narratives Classification Report:")
    print(best_classification_report_sub)

In [76]:
target_names_nar = mlb_narratives.classes_
target_names_sub = mlb_subnarratives.classes_

evaluate_model(
    model=model,
    embeddings=val_embeddings_tensor,
    y_nar_true=y_val_nar,
    y_sub_nar_true=y_val_sub_nar,
    target_names_nar=target_names_nar,
    target_names_sub=target_names_sub
)

Best Threshold: 0.6, Best F1 Score: 0.3765128675667295

Best Narratives Classification Report:
                                                   precision    recall  f1-score   support

                         Amplifying Climate Fears       0.79      1.00      0.88        45
                     Amplifying war-related fears       0.67      0.67      0.67        48
Blaming the war on others rather than the invader       0.38      0.62      0.47        37
                     Climate change is beneficial       0.33      0.50      0.40         4
             Controversy about green technologies       0.26      0.83      0.40         6
                    Criticism of climate movement       0.45      0.67      0.54        15
                    Criticism of climate policies       0.35      0.71      0.47        17
        Criticism of institutions and authorities       0.58      0.81      0.68        32
                             Discrediting Ukraine       0.60      0.68      0.64     

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to cont

In [77]:
import torch

save_path = "multi_task_model_eval.pth"

torch.save({
    'model_state_dict': model.state_dict(),
    'input_size': 768,
    'hidden_size': 128,
    'num_narratives': len(mlb_narratives.classes_),
    'num_subnarratives': len(mlb_subnarratives.classes_),
    'dropout_rate': 0.3
}, save_path)

print(f"Model saved to {save_path}")

Model saved to multi_task_model_eval.pth


In [78]:
mlb_narratives.classes_

array(['Amplifying Climate Fears', 'Amplifying war-related fears',
       'Blaming the war on others rather than the invader',
       'Climate change is beneficial',
       'Controversy about green technologies',
       'Criticism of climate movement', 'Criticism of climate policies',
       'Criticism of institutions and authorities',
       'Discrediting Ukraine', 'Discrediting the West, Diplomacy',
       'Distrust towards Media', 'Downplaying climate change',
       'Green policies are geopolitical instruments',
       'Hidden plots by secret schemes of powerful groups',
       'Negative Consequences for the West', 'Other',
       'Overpraising the West', 'Praise of Russia',
       'Questioning the measurements and science', 'Russia is the Victim',
       'Speculating war outcomes'], dtype=object)

In [79]:
import joblib

joblib.dump(mlb_narratives, 'mlb_narratives.pkl')
joblib.dump(mlb_subnarratives, 'mlb_subnarratives.pkl')

['mlb_subnarratives.pkl']

### Fine-tuning Roberta to predict narratives

In [ ]:
from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_data(entity_contexts, max_length=512):
    encodings = tokenizer(entity_contexts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    return encodings

train_encodings = tokenize_data(dataset_train['content'].tolist())
val_encodings = tokenize_data(dataset_val['content'].tolist())

In [ ]:
train_narrative_truths = dataset_train['narratives_encoded'].tolist()

train_narrative_truths[:5]


val_narrative_truths = dataset_val['narratives_encoded'].tolist()

val_narrative_truths[:5]

In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

model_config = {
    'batch_size': 8,
    'num_epochs': 10,
    'lr': 3e-5,
}

In [ ]:
from transformers import XLMRobertaModel

class NarrativeClassificationRoberta(nn.Module):
    def __init__(self, narrative_classes):
        super(NarrativeClassificationRoberta, self).__init__()
        self.narrative_classes = narrative_classes
        self.backbone = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        self.classifier = nn.Linear(self.backbone.config.hidden_size, narrative_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]

        narrative_logits = self.classifier(pooled_output)

        return narrative_logits

narrative_classifier_roberta = NarrativeClassificationRoberta(narrative_classes=len(mlb_narratives.classes_))

In [ ]:
class NarrativesDataset(Dataset):
    def __init__(self, encodings, narrative_labels):
        self.encodings = encodings
        self.narrative_labels = narrative_labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['narrative_labels'] = torch.tensor(self.narrative_labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.narrative_labels)

train_dataset_nar = NarrativesDataset(
    train_encodings,
    dataset_train['narratives_encoded'].tolist(),
)

val_dataset_nar = NarrativesDataset(
    val_encodings,
    dataset_val['narratives_encoded'].tolist(),
)

train_loader_nar = DataLoader(train_dataset_nar, batch_size=model_config['batch_size'], shuffle=True)
val_loader_nar = DataLoader(val_dataset_nar, batch_size=model_config['batch_size'], shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer_nar = optim.AdamW(narrative_classifier_roberta.parameters(), lr=model_config['lr'])

narrative_classifier_roberta.to(device)

In [ ]:
from torch.nn.functional import binary_cross_entropy_with_logits

def bce_with_weights(outputs, targets, weights=None):
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none')
    loss = criterion(outputs, targets)

    if weights is not None:
        weights = weights.to(outputs.device)
        loss = loss * weights

    loss = loss.mean()
    return loss, loss.item()

In [ ]:
from transformers import get_linear_schedule_with_warmup

total_steps = len(train_loader_nar) * model_config['num_epochs']
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(optimizer_nar, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

In [ ]:
import torch
import pandas as pd

def compute_class_weights(dataset, label_column, classes, max_weight=10, epsilon=1e-8):
    label_df = pd.DataFrame(dataset[label_column].tolist(), columns=classes)
    class_counts = label_df.sum().sort_values(ascending=False)
    class_counts = class_counts.reindex(classes)

    total_samples = len(dataset)

    class_weights = {
        i: total_samples / (len(class_counts) * (count + epsilon))
        for i, count in enumerate(class_counts.values)
    }

    weights_tensor = torch.tensor(list(class_weights.values()), dtype=torch.float)
    weights_tensor = torch.clamp(weights_tensor, max=max_weight)

    for i, class_name in enumerate(classes):
        print(f"Class: {class_name}, Weight: {weights_tensor[i].item()}")

    return weights_tensor

narrative_weights_tensor = compute_class_weights(
    dataset=dataset_train,
    label_column='narratives_encoded',
    classes=mlb_narratives.classes_
)

In [ ]:
def freeze_layers(model, num_layers_to_freeze=2):
  assert hasattr(model, 'classifier'), "Model must have a classifier attribute"
  for i in range(num_layers_to_freeze):
      for param in model.backbone.encoder.layer[i].parameters():
          param.requires_grad = False

  for param in model.classifier.parameters():
      param.requires_grad = True

In [ ]:
freeze_layers(narrative_classifier_roberta, num_layers_to_freeze=1)

In [ ]:
def train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    label_column,
    scheduler=None,
    weights=None,
    num_epochs=model_config['num_epochs'],
    device='cuda'
):
    for epoch in range(num_epochs):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch[label_column].to(device)

            logits = model(input_ids, attention_mask)
            loss, loss_item = bce_with_weights(logits, labels, weights)

            loss.backward()
            optimizer.step()
            if scheduler:
              scheduler.step()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch[label_column].to(device)

                logits = model(input_ids, attention_mask)
                _, batch_loss = bce_with_weights(logits, labels)
                val_loss += batch_loss

        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch + 1}, Training Loss (AVG): {loss_item}, Validation Loss (AVG): {avg_val_loss}")

In [ ]:
train_model(
    model=narrative_classifier_roberta,
    train_loader=train_loader_nar,
    val_loader=val_loader_nar,
    optimizer=optimizer_nar,
    scheduler=scheduler,
    label_column='narrative_labels',
    num_epochs=model_config.get('num_epochs', 5),
    device=device
)

In [ ]:
from sklearn.metrics import f1_score

def evaluate_transformer_model(
    model,
    val_loader,
    label_column,
    thresholds=np.arange(0.1, 1.0, 0.1),
    device='cuda',
    target_names=None
):
    best_threshold = 0
    best_f1 = 0
    best_classification_report = None

    all_preds = []
    all_truths = []

    for threshold in thresholds:
        preds = []
        truths = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch[label_column].to(device)
                logits = model(input_ids, attention_mask)
                probs = torch.sigmoid(logits)

                model_preds = (probs >= threshold).int()
                preds.extend(model_preds.cpu().numpy())
                truths.extend(labels.cpu().numpy())

        classification_rep = classification_report(truths, preds, target_names=target_names, zero_division=0)
        current_f1 = f1_score(truths, preds, average='macro')

        if current_f1 > best_f1:
            best_f1 = current_f1
            best_threshold = threshold
            best_classification_report = classification_rep

    print(f"Best Threshold: {best_threshold}, Best F1 Score: {best_f1}")
    print("Best Classification Report:")
    print(best_classification_report)

In [ ]:
evaluate_transformer_model(
    narrative_classifier_roberta,
    val_loader_nar,
    label_column='narrative_labels',
    target_names=mlb_narratives.classes_
)

### Fine-tuning Roberta to predict subnarratives

In [ ]:
train_sub_narrative_truths = dataset_train['subnarratives_encoded'].tolist()

train_sub_narrative_truths[0][:5]

In [ ]:
val_sub_narrative_truths = dataset_val['subnarratives_encoded'].tolist()

val_sub_narrative_truths[0][:5]

In [ ]:
import torch
from torch import nn
from transformers import XLMRobertaModel

class SubNarrativeClassificationRoberta(nn.Module):
    def __init__(self, sub_narrative_classes):
        super(SubNarrativeClassificationRoberta, self).__init__()
        self.sub_narrative_classes = sub_narrative_classes
        self.backbone = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        self.classifier = nn.Linear(self.backbone.config.hidden_size, sub_narrative_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]

        subnarrative_logits = self.classifier(pooled_output)


        return subnarrative_logits

subnarrative_classifier_roberta = SubNarrativeClassificationRoberta(sub_narrative_classes=len(mlb_subnarratives.classes_))

In [ ]:
import torch
from torch.utils.data import Dataset

class SubNarrativesDataset(Dataset):
    def __init__(self, encodings, sub_narrative_labels):
        self.encodings = encodings
        self.sub_narrative_labels = sub_narrative_labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['sub_narrative_labels'] = torch.tensor(self.sub_narrative_labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.sub_narrative_labels)

train_dataset = SubNarrativesDataset(
    train_encodings,
    dataset_train['subnarratives_encoded'].tolist(),
)

val_dataset = SubNarrativesDataset(
    val_encodings,
    dataset_val['subnarratives_encoded'].tolist(),
)

train_loader_sub = DataLoader(train_dataset, batch_size=model_config['batch_size'], shuffle=True)
val_loader_sub = DataLoader(val_dataset, batch_size=model_config['batch_size'], shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer_sub = optim.AdamW(subnarrative_classifier_roberta.parameters(), lr=model_config['lr'])
criterion_sub = nn.BCEWithLogitsLoss()

subnarrative_classifier_roberta.to(device)

In [ ]:
freeze_layers(subnarrative_classifier_roberta, num_layers_to_freeze=1)

In [ ]:
train_model(subnarrative_classifier_roberta, train_loader_sub, val_loader_sub, optimizer_sub, label_column='sub_narrative_labels')

In [ ]:
evaluate_transformer_model(subnarrative_classifier_roberta, val_loader_sub, label_column='sub_narrative_labels', target_names=mlb_subnarratives.classes_)

### Predicting narratives and subnarratives using MultiTask learning

In [ ]:
from torch import nn
from transformers import XLMRobertaModel

class MultiTaskRoberta(nn.Module):
    def __init__(self, narrative_classes, sub_narrative_classes):
        super(MultiTaskRoberta, self).__init__()
        self.backbone = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        self.narrative_classifier = nn.Linear(self.backbone.config.hidden_size, narrative_classes)
        self.sub_narrative_classifier = nn.Linear(self.backbone.config.hidden_size, sub_narrative_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]

        narrative_logits = self.narrative_classifier(pooled_output)

        sub_narrative_logits = self.sub_narrative_classifier(pooled_output)

        return narrative_logits, sub_narrative_logits

multi_task_model = MultiTaskRoberta(narrative_classes=len(mlb_narratives.classes_), sub_narrative_classes=len(mlb_subnarratives.classes_))

In [ ]:
from torch.utils.data import Dataset

class NewsDataset(Dataset):
    def __init__(self, encodings, narrative_labels, subnarrative_labels):
        self.encodings = encodings
        self.narrative_labels = narrative_labels
        self.subnarrative_labels = subnarrative_labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['narrative_labels'] = torch.tensor(self.narrative_labels[idx], dtype=torch.float)
        item['subnarrative_labels'] = torch.tensor(self.subnarrative_labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.narrative_labels)

train_dataset = NewsDataset(
    train_encodings,
    dataset_train['narratives_encoded'].tolist(),
    dataset_train['subnarratives_encoded'].tolist()
)

val_dataset = NewsDataset(
    val_encodings,
    dataset_val['narratives_encoded'].tolist(),
    dataset_val['subnarratives_encoded'].tolist()
)

train_loader = DataLoader(train_dataset, batch_size=model_config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=model_config['batch_size'], shuffle=False)

In [ ]:
def mask_sub_narrative_logits(narrative_probs, sub_narrative_logits, narrative_to_sub_mapping, threshold=0.2):
    batch_size, sub_narrative_size = sub_narrative_logits.size()
    mask = torch.zeros_like(sub_narrative_logits)

    for i in range(batch_size):
        active_narratives = torch.where(narrative_probs[i] > threshold)[0]

        for narrative_idx in active_narratives:
            sub_indices = list(narrative_to_sub_mapping[narrative_idx.item()])
            mask[i, sub_indices] = 1

    epsilon = 1e-8
    masked_sub_narrative_logits = sub_narrative_logits * mask + epsilon * (1 - mask)

    return masked_sub_narrative_logits

In [ ]:
narrative_to_sub_mapping = {}

for i, row in dataset_train.iterrows():
    narratives_encoded_list = row['narratives_encoded']
    subnarratives_encoded_list = row['subnarratives_encoded']

    for j in range(len(narratives_encoded_list)):
        if narratives_encoded_list[j] == 1:
            if j not in narrative_to_sub_mapping:
                narrative_to_sub_mapping[j] = set()

            subnarratives_encoded_list = np.asarray(subnarratives_encoded_list)

            indices_of_ones = np.where(subnarratives_encoded_list == 1)[0]

            narrative_to_sub_mapping[j].update(indices_of_ones)

In [ ]:
from torch.nn.functional import binary_cross_entropy_with_logits

def compute_loss(narrative_logits, sub_narrative_logits, narrative_labels, sub_narrative_labels, narrative_to_sub_mapping, weights=None):
    narrative_loss = binary_cross_entropy_with_logits(narrative_logits, narrative_labels, pos_weight=weights)

    masked_sub_logits = mask_sub_narrative_logits(narrative_labels, sub_narrative_logits, narrative_to_sub_mapping)
    sub_narrative_loss = binary_cross_entropy_with_logits(masked_sub_logits, sub_narrative_labels)

    return narrative_loss + sub_narrative_loss

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.AdamW(multi_task_model.parameters(), lr=model_config['lr'])

multi_task_model.to(device)

In [ ]:
def freeze_multi_task_roberta_layers(model, num_layers_to_freeze=2):
  for i in range(num_layers_to_freeze):
    for param in model.backbone.encoder.layer[i].parameters():
      param.requires_grad = False

  for param in model.narrative_classifier.parameters():
    param.requires_grad = True

  for param in model.sub_narrative_classifier.parameters():
    param.requires_grad = True

In [ ]:
freeze_multi_task_roberta_layers(multi_task_model)

In [ ]:
for epoch in range(5):
    multi_task_model.train()
    epoch_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        narrative_labels = batch['narrative_labels'].to(device)
        sub_narrative_labels = batch['subnarrative_labels'].to(device)

        narrative_logits, sub_narrative_logits = multi_task_model(input_ids, attention_mask)
        loss = compute_loss(narrative_logits, sub_narrative_logits, narrative_labels, sub_narrative_labels, narrative_to_sub_mapping)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = epoch_loss / len(train_loader)

    multi_task_model.eval()
    val_loss = 0
    for batch in val_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            narrative_labels = batch['narrative_labels'].to(device)
            sub_narrative_labels = batch['subnarrative_labels'].to(device)

            narrative_logits, sub_narrative_logits = multi_task_model(input_ids, attention_mask)
            loss = compute_loss(narrative_logits, sub_narrative_logits, narrative_labels, sub_narrative_labels, narrative_to_sub_mapping)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch + 1}, Training Loss (AVG): {avg_train_loss:.4f}, Validation Loss (AVG): {avg_val_loss:.4f}")

In [ ]:
def evaluate_with_threshold_selection(
    model,
    val_loader,
    narrative_targets_name,
    subnarrative_targets_name,
    thresholds=np.arange(0.1, 1.0, 0.1),
    device='cuda'
):
    best_narrative_threshold = 0
    best_subnarrative_threshold = 0
    best_narrative_f1 = 0
    best_subnarrative_f1 = 0
    best_narrative_report = None
    best_subnarrative_report = None

    all_narratives_truth = []
    all_subnarratives_truth = []

    for batch in val_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            narrative_labels = batch['narrative_labels'].to(device)
            subnarrative_labels = batch['subnarrative_labels'].to(device)
            narrative_logits, subnarrative_logits = model(input_ids, attention_mask)

            narrative_probs = torch.sigmoid(narrative_logits)
            subnarrative_probs = torch.sigmoid(subnarrative_logits)

            all_narratives_truth.extend(narrative_labels.cpu().numpy())
            all_subnarratives_truth.extend(subnarrative_labels.cpu().numpy())

    for threshold in thresholds:
        narratives_pred = []
        subnarratives_pred = []

        for batch in val_loader:
            with torch.no_grad():
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                narrative_logits, subnarrative_logits = model(input_ids, attention_mask)

                narrative_probs = torch.sigmoid(narrative_logits)
                subnarrative_probs = torch.sigmoid(subnarrative_logits)

                narratives_pred.extend((narrative_probs >= threshold).int().cpu().numpy())
                subnarratives_pred.extend((subnarrative_probs >= threshold).int().cpu().numpy())

        narrative_report = classification_report(
            all_narratives_truth,
            narratives_pred,
            target_names=narrative_targets_name,
            zero_division=0
        )
        narrative_f1 = f1_score(all_narratives_truth, narratives_pred, average='macro')

        if narrative_f1 > best_narrative_f1:
            best_narrative_f1 = narrative_f1
            best_narrative_threshold = threshold
            best_narrative_report = narrative_report

        subnarrative_report = classification_report(
            all_subnarratives_truth,
            subnarratives_pred,
            target_names=subnarrative_targets_name,
            zero_division=0
        )
        subnarrative_f1 = f1_score(all_subnarratives_truth, subnarratives_pred, average='macro')

        if subnarrative_f1 > best_subnarrative_f1:
            best_subnarrative_f1 = subnarrative_f1
            best_subnarrative_threshold = threshold
            best_subnarrative_report = subnarrative_report

    print(f"Best Narrative Threshold: {best_narrative_threshold}, Best Narrative F1 Score: {best_narrative_f1}")
    print("Best Classification Report for Narratives:")
    print(best_narrative_report)

    print(f"Best Subnarrative Threshold: {best_subnarrative_threshold}, Best Subnarrative F1 Score: {best_subnarrative_f1}")
    print("Best Classification Report for Subnarratives:")
    print(best_subnarrative_report)

evaluate_with_threshold_selection(
    model=multi_task_model,
    val_loader=val_loader,
    narrative_targets_name=mlb_narratives.classes_,
    subnarrative_targets_name=mlb_subnarratives.classes_,
    thresholds=np.arange(0.1, 1.0, 0.1),
    device=device
)